In [27]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
url = "https://en.wikipedia.org/wiki/RuPaul%27s_Drag_Race_terminology"

In [4]:
response = requests.get(url)
soup = bs(response.text)

In [24]:
wikitable= soup.find("table", class_="wikitable")
wikitable = wikitable.findAll("tr")
filtered_wikitable = list(filter( lambda x: len(x.findAll("td"))==2 ,wikitable))
words = [ {'term': re.sub("\[[0-9]+\]", "" ,row.findAll("td")[0].text), 'definition': re.sub("\[[0-9]+\]","",row.findAll("td")[1].text.strip())} for row in filtered_wikitable]
words =list(filter( lambda x: x['definition']!="", words))
wordsOk = list(filter(lambda x: not "/" in x['term'] , words))
wordsOk
wordsToSplit = list(filter(lambda x: "/" in x['term'] , words))
wordsToSplit

[{'term': "beat / beat one's face",
  'definition': "to blend one's makeup, often with a sponge"},
 {'term': 'best Judies / Judies',
  'definition': "a gay man's gay best friend, usually platonically; the name of an All Stars season 4 makeover challenge"},
 {'term': 'chanté, you stay / shantay, you stay / shante, you stay',
  'definition': 'phrase used by RuPaul to declare then winner of a lip sync'},
 {'term': 'drag mother / dragmother',
  'definition': 'an established drag queen who mentors a new queen, her "daughter" how to get started; many queens use the same last name as their drag mother, creating "family" lineages, sometimes called "houses"'},
 {'term': 'fish / fishy',
  'definition': 'a term used to describe when a drag queen looks like a cis-gendered woman'},
 {'term': 'gag / gagging',
  'definition': 'another term used in place of "stunned"'},
 {'term': 'read / reading',
  'definition': 'verbally insulting another queen about their acts, looks, or persona, often in jest; the

In [31]:
newWordsOk = []
for word in wordsToSplit:
    terms = re.findall("([^/]+)", word["term"])
    newWordsOk += [ {'term': term.strip(), 'definition': word['definition']}  for term in terms]
words = wordsOk + newWordsOk 
words

[{'term': 'apolo-lie',
  'definition': "a fake apology, coined by Widow Von'Du (season 12)"},
 {'term': 'bam',
  'definition': 'a joyous expression used by Alexis Mateo (season 3; All Stars seasons 1 and 5), originally for season 3\'s "Life, Liberty & the Pursuit of Style" maxi challenge and later repeated by RuPaul'},
 {'term': 'big girl',
  'definition': 'a drag queen who wears plus-size clothing'},
 {'term': 'busted', 'definition': 'being unpolished or messy'},
 {'term': 'charisma, uniqueness, nerve, and talent',
  'definition': 'euphemism for C.U.N.T, shown by the first letter of each word'},
 {'term': 'chicken cutlets',
  'definition': 'gel inserts used to create cleavage'},
 {'term': 'choices',
  'definition': 'a catchphrase used by Tatianna (season 2; All Stars season 2)'},
 {'term': 'condragulations',
  'definition': 'drag wordplay for the word "congratulations"'},
 {'term': 'cooking', 'definition': 'allowing time for face powder to set'},
 {'term': 'country realness',
  'defin

In [28]:
analyzer = SentimentIntensityAnalyzer()

In [40]:
addToLexiton =  { word['term']: analyzer.polarity_scores(word["definition"])['compound'] for word in words }
newAddToLexiton = {}
for key in addToLexiton:
    if addToLexiton[key] == 0.0:
        newAddToLexiton[key]=input(key)
    else:
        newAddToLexiton[key]=addToLexiton[key]

charisma, uniqueness, nerve, and talent 1
choices -0.5
cooking 0.0
country realness 0.0
cucu 0.3
echa pa lante 0.0
interior illusions lounge -0.8
leotarded 0.0
lip sync for your life -0.2
sashay away -0.7
serve 1
sexcretary 0.0
sickening 1.0
beat 0.5
beat one's face .1


In [45]:
import json
with open('Data/addToLexiton.json', 'w') as f:
    json.dump(newAddToLexiton, f)